# Installing required libraries

In [1]:
!conda install polars -y -q

Channels:
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.



# Importing required libraries

In [2]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

# Config file (Paths of datasets to be joined)

In [3]:
class config:
    target_path = './Determine_modeling_datasets/Determine_modeling_age_group_outcome_data.parquet'
    demo_path = './Determine_modeling_datasets/Determine_modeling_demographics_data.parquet'
    med_path = './Determine_modeling_datasets/Determine_modeling_medications_thrs_50.parquet'
    diag_path = './Determine_modeling_datasets/Determine_modeling_diagnoses_thrs_50_without_icd10z.parquet'
    lab_path = './Determine_modeling_datasets/Determine_modeling_labresults_latest_thrs_50.parquet'
    bmi_path = 'Determine_modeling_bmi_data_manual.parquet'
    dia_bp_path = 'Determine_modeling_diastolic_bp_data.parquet'
    sys_bp_path = 'Determine_modeling_systolic_bp_data.parquet'
    cvs_path = './Determine_modeling_datasets/Determine_cvs_pivot.parquet'
    
    use_bmi_bp = True
    use_cvs = False
    

# Joining the datasets

In [4]:
target_df = pl.scan_parquet(config.target_path)
demo_df = pl.scan_parquet(config.demo_path)
med_df = pl.scan_parquet(config.med_path)
diag_df = pl.scan_parquet(config.diag_path)
lab_df = pl.scan_parquet(config.lab_path)

bmi_df = pl.scan_parquet(config.bmi_path)
dia_bp_df = pl.scan_parquet(config.dia_bp_path)
sys_bp_df = pl.scan_parquet(config.sys_bp_path)
cvs_df = pl.scan_parquet(config.cvs_path)

In [5]:
diag_df.collect().head()

PATIENT_NUM,611.0,790.6,250.42,289.4,591.0,585.3,401.1,338.2,272.1,261.4,112.3,512.9,513.31,605.0,495.0,574.1,455.0,563.0,38.0,172.21,261.2,300.1,355.1,458.1,458.9,480.0,994.2,599.9,600.0,745.0,689.0,496.2,110.11,215.0,218.1,285.0,…,151.0,695.22,441.1,ICD10CM:W00,504.1,ICD10CM:J45,286.12,ICD10CM:W11,704.12,732.0,709.5,441.2,751.2,227.2,613.0,270.34,622.0,ICD10CM:R68,ICD10CM:G40,ICD10CM:N76,ICD10CM:F51,ICD10CM:B18,ICD10CM:G89,ICD10CM:L02,ICD10CM:R63,ICD10CM:N93,ICD10CM:N95,ICD10CM:R39,ICD10CM:E04,ICD10CM:E87,ICD10CM:N94,ICD10CM:K92,ICD10CM:F34,ICD10CM:R92,ICD10CM:R79,ICD10CM:A74,ICD10CM:R26
i64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
556,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
562,0,0,0,0,0,0,1,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
621,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
624,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
642,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
'1010.0' in diag_df.collect().columns

False

In [7]:
print("Number of unique patient records in demographics dataframe: ", len(np.unique(demo_df.select('Patient_num')
                                                                                    .collect()['Patient_num'].to_list())))
print("Number of unique patient records in diagnoses dataframe: ", len(np.unique(diag_df.select('PATIENT_NUM')
                                                                                    .collect()['PATIENT_NUM'].to_list())))
print("Number of unique patient records in medications dataframe: ", len(np.unique(med_df.select('PATIENT_NUM')
                                                                                    .collect()['PATIENT_NUM'].to_list())))
print("Number of unique patient records in labresults dataframe: ", len(np.unique(lab_df.select('PATIENT_NUM')
                                                                                    .collect()['PATIENT_NUM'].to_list())))
print("Number of unique patient records in BMI dataframe: ", len(np.unique(bmi_df.select('PATIENT_NUM')
                                                                                    .collect()['PATIENT_NUM'].to_list())))
print("Number of unique patient records in Dia bp dataframe: ", len(np.unique(dia_bp_df.select('PATIENT_NUM')
                                                                                    .collect()['PATIENT_NUM'].to_list())))
print("Number of unique patient records in Sys dataframe: ", len(np.unique(sys_bp_df.select('PATIENT_NUM')
                                                                                    .collect()['PATIENT_NUM'].to_list())))

Number of unique patient records in demographics dataframe:  1068408
Number of unique patient records in diagnoses dataframe:  818322
Number of unique patient records in medications dataframe:  598798
Number of unique patient records in labresults dataframe:  533819
Number of unique patient records in BMI dataframe:  880276
Number of unique patient records in Dia bp dataframe:  924831
Number of unique patient records in Sys dataframe:  924262


In [8]:
med_pids = np.unique(med_df.select(['PATIENT_NUM']).collect()['PATIENT_NUM'].to_list())
dx_pids = np.unique(diag_df.select(['PATIENT_NUM']).collect()['PATIENT_NUM'].to_list())
lab_pid = np.unique(lab_df.select(['PATIENT_NUM']).collect()['PATIENT_NUM'].to_list())

# Convert lists to sets and use the union operation
result = set(med_pids) | set(dx_pids) | set(lab_pid)

# Convert the set back to a list if needed
result_list = list(result)

# Display the result
print("Number of patient ids that have data in atleast one of the medications, lab_results and diagnoses: ",len(result_list))


Number of patient ids that have data in atleast one of the medications, lab_results and diagnoses:  968392


In [9]:
target_df_filtered = target_df.filter(pl.col('PATIENT_NUM').is_in(result_list))
demo_df_filtered = demo_df.filter(pl.col('Patient_num').is_in(result_list))

In [10]:
temp_df = target_df_filtered.join(demo_df_filtered, how = 'left', left_on = 'PATIENT_NUM', right_on = 'Patient_num')

In [11]:
temp_df.columns

/tmp/ipykernel_21797/3421553128.py:1: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  temp_df.columns


['PATIENT_NUM',
 'Age_group',
 'FirstOutcomeDate',
 'Outcome',
 'Sex_CD',
 'Race_CD',
 'Hispanic_CD',
 'Gender_CD']

In [12]:
temp_df = temp_df.join(med_df, on = 'PATIENT_NUM', how = 'left')
temp_df = temp_df.fill_null(0)

In [13]:
temp_df.head().collect()

PATIENT_NUM,Age_group,FirstOutcomeDate,Outcome,Sex_CD,Race_CD,Hispanic_CD,Gender_CD,nitrofurantoin,nystatin,atorvastatin,amlodipine,lisinopril_hydrochlorothiazide,tadalafil,albuterol,chlorthalidone,potassium chloride,vilanterol_fluticasone,hydrocortisone,lisinopril,"sennosides, USP_docusate",mupirocin,folic acid,tamsulosin,meloxicam,amitriptyline,quetiapine,gabapentin,tranexamic acid,polyvinyl alcohol,terbinafine,ascorbic acid,ferrous sulfate,losartan,clavulanate_amoxicillin,penicillin V,duloxetine,…,1291624,benzoyl peroxide_erythromycin,tetrahydrozoline,sofosbuvir_ledipasvir,acetaminophen_dextromethorphan_phenylephrine,isosorbide dinitrate,nonoxynol-9,betamethasone_calcipotriene,hyaluronate,indapamide,phenylephrine_promethazine,tazarotene,calamine_pramoxine,clomipramine,1359572,potassium nitrate_sodium fluoride,sodium polystyrene sulfonate,lidocaine_menthol,meningococcal group B vaccine,chlorpheniramine_dextromethorphan,wheat dextrin,etravirine,guaifenesin_phenylephrine,letrozole,crisaborole,isotretinoin,acetaminophen_diphenhydramine,salmon calcitonin,1302488,phendimetrazine,collagenase,282708,povidone-iodine,naftifine,codeine,triprolidine_pseudoephedrine,1303868
i64,str,datetime[μs],i64,str,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
10464243,"""18-34""",null,0,"""F""","""UN""","""R""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2245024,"""45-54""",null,0,"""F""","""05""","""N""","""UN""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8474922,"""55-64""",null,0,"""F""","""03""","""N""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7621798,"""18-34""",null,0,"""F""","""05""","""Y""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10756244,"""18-34""",null,0,"""M""","""05""","""N""","""M""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
temp_df = temp_df.join(diag_df, on = 'PATIENT_NUM', how = 'left')
temp_df = temp_df.fill_null(0)

In [15]:
temp_df.head().collect()

PATIENT_NUM,Age_group,FirstOutcomeDate,Outcome,Sex_CD,Race_CD,Hispanic_CD,Gender_CD,nitrofurantoin,nystatin,atorvastatin,amlodipine,lisinopril_hydrochlorothiazide,tadalafil,albuterol,chlorthalidone,potassium chloride,vilanterol_fluticasone,hydrocortisone,lisinopril,"sennosides, USP_docusate",mupirocin,folic acid,tamsulosin,meloxicam,amitriptyline,quetiapine,gabapentin,tranexamic acid,polyvinyl alcohol,terbinafine,ascorbic acid,ferrous sulfate,losartan,clavulanate_amoxicillin,penicillin V,duloxetine,…,151.0,695.22,441.1,ICD10CM:W00,504.1,ICD10CM:J45,286.12,ICD10CM:W11,704.12,732.0,709.5,441.2,751.2,227.2,613.0,270.34,622.0,ICD10CM:R68,ICD10CM:G40,ICD10CM:N76,ICD10CM:F51,ICD10CM:B18,ICD10CM:G89,ICD10CM:L02,ICD10CM:R63,ICD10CM:N93,ICD10CM:N95,ICD10CM:R39,ICD10CM:E04,ICD10CM:E87,ICD10CM:N94,ICD10CM:K92,ICD10CM:F34,ICD10CM:R92,ICD10CM:R79,ICD10CM:A74,ICD10CM:R26
i64,str,datetime[μs],i64,str,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
10464243,"""18-34""",null,0,"""F""","""UN""","""R""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2245024,"""45-54""",null,0,"""F""","""05""","""N""","""UN""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8474922,"""55-64""",null,0,"""F""","""03""","""N""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7621798,"""18-34""",null,0,"""F""","""05""","""Y""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10756244,"""18-34""",null,0,"""M""","""05""","""N""","""M""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
temp_df = temp_df.join(lab_df, on = 'PATIENT_NUM', how = 'left')
temp_df = temp_df.fill_null(-100)

In [17]:
temp_df = temp_df.with_columns([
    pl.col('Sex_CD').str.replace(r'^(NI|OT|UN)$', 'UNK', literal=False).alias('Sex_CD'),
    pl.col('Race_CD').str.replace(r'^(NI|OT|UN)$', 'UNK', literal=False).alias('Race_CD'),
    pl.col('Hispanic_CD').str.replace(r'^(R|NI|UN)$', 'UNK', literal=False).alias('Hispanic_CD'),
    pl.col('Gender_CD').str.replace(r'^(OT|NI|UN)$', 'UNK', literal=False).alias('Gender_CD')
])

In [18]:
if config.use_bmi_bp:
    temp_df = temp_df.join(bmi_df, on = 'PATIENT_NUM', how = 'left')
    temp_df = temp_df.fill_null(-100)
    temp_df = temp_df.join(dia_bp_df, on = 'PATIENT_NUM', how = 'left')
    temp_df = temp_df.fill_null(-100)
    temp_df = temp_df.join(sys_bp_df, on = 'PATIENT_NUM', how = 'left')
    temp_df = temp_df.fill_null(-100)
    

In [19]:
if config.use_cvs:
    temp_df = temp_df.join(cvs_df, on = 'PATIENT_NUM',how ='left')
    temp_df = temp_df.fill_null(-100)

In [20]:
temp_df.head().collect()

PATIENT_NUM,Age_group,FirstOutcomeDate,Outcome,Sex_CD,Race_CD,Hispanic_CD,Gender_CD,nitrofurantoin,nystatin,atorvastatin,amlodipine,lisinopril_hydrochlorothiazide,tadalafil,albuterol,chlorthalidone,potassium chloride,vilanterol_fluticasone,hydrocortisone,lisinopril,"sennosides, USP_docusate",mupirocin,folic acid,tamsulosin,meloxicam,amitriptyline,quetiapine,gabapentin,tranexamic acid,polyvinyl alcohol,terbinafine,ascorbic acid,ferrous sulfate,losartan,clavulanate_amoxicillin,penicillin V,duloxetine,…,LOINC:42250-1,LOINC:43743-4,LOINC:21525-1,LOINC:2988-4,LOINC:2021-4,LOINC:4575-7,LOINC:48343-8,LOINC:83122-2,LOINC:9749-3,LOINC:9321-1,LOINC:76630-3,LOINC:2601-3,LOINC:53765-4,LOINC:4547-6,LOINC:21458-5,LOINC:18184-2,LOINC:19195-7,LOINC:14805-6,LOINC:13056-7,LOINC:20567-4,LOINC:6942-7,LOINC:1764-0,LOINC:41649-5,LOINC:50210-4,LOINC:83102-4,LOINC:72888-1,LOINC:2349-9,LOINC:2657-5,mode_height,average_weight,BMI,average_diastolic_value,min_diastolic_value,max_diastolic_value,average_systolic_value,min_systolic_value,max_systolic_value
i64,str,datetime[μs],i64,str,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,"decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]",f32,f32,f64,"decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]"
10464243,"""18-34""",null,0,"""F""","""UNK""","""UNK""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,69.0,172.333328,25.446404,-100.00000,81.00000,86.00000,-100.00000,128.00000,132.00000
2245024,"""45-54""",null,0,"""F""","""05""","""N""","""UNK""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.0,-100.0,-100.0,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000
8474922,"""55-64""",null,0,"""F""","""03""","""N""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,68.0,210.0,31.926904,-100.00000,70.00000,84.00000,-100.00000,122.00000,138.00000
7621798,"""18-34""",null,0,"""F""","""05""","""Y""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,59.5,135.800003,26.966289,-100.00000,80.00000,80.00000,-100.00000,117.00000,117.00000
10756244,"""18-34""",null,0,"""M""","""05""","""N""","""M""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.000

In [21]:
np.unique(temp_df.select('Outcome').collect()['Outcome'].to_list(), return_counts = True)

(array([0, 1]), array([922506,  45832]))

In [22]:
np.unique(temp_df.select('Gender_CD').collect()['Gender_CD'].to_list(), return_counts = True)

(array(['GQ', 'M', 'TG', 'UNK', 'W'], dtype='<U3'),
 array([  2299, 283490,   4095, 202363, 476091]))

In [23]:
# Dicts to map 
age_group_dict = {'18-34': 0,
                  '35-44': 1,
                  '45-54':2,
                  '55-64':3,
                  '65-74':4,
                  '75_older':5    
}

# nominal encoding Gender, Sex and Race


In [24]:
temp_df = temp_df.with_columns(pl.col('Age_group').replace_strict(age_group_dict).alias('Age_group'))

In [25]:
temp_df.head().collect()

PATIENT_NUM,Age_group,FirstOutcomeDate,Outcome,Sex_CD,Race_CD,Hispanic_CD,Gender_CD,nitrofurantoin,nystatin,atorvastatin,amlodipine,lisinopril_hydrochlorothiazide,tadalafil,albuterol,chlorthalidone,potassium chloride,vilanterol_fluticasone,hydrocortisone,lisinopril,"sennosides, USP_docusate",mupirocin,folic acid,tamsulosin,meloxicam,amitriptyline,quetiapine,gabapentin,tranexamic acid,polyvinyl alcohol,terbinafine,ascorbic acid,ferrous sulfate,losartan,clavulanate_amoxicillin,penicillin V,duloxetine,…,LOINC:42250-1,LOINC:43743-4,LOINC:21525-1,LOINC:2988-4,LOINC:2021-4,LOINC:4575-7,LOINC:48343-8,LOINC:83122-2,LOINC:9749-3,LOINC:9321-1,LOINC:76630-3,LOINC:2601-3,LOINC:53765-4,LOINC:4547-6,LOINC:21458-5,LOINC:18184-2,LOINC:19195-7,LOINC:14805-6,LOINC:13056-7,LOINC:20567-4,LOINC:6942-7,LOINC:1764-0,LOINC:41649-5,LOINC:50210-4,LOINC:83102-4,LOINC:72888-1,LOINC:2349-9,LOINC:2657-5,mode_height,average_weight,BMI,average_diastolic_value,min_diastolic_value,max_diastolic_value,average_systolic_value,min_systolic_value,max_systolic_value
i64,i64,datetime[μs],i64,str,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,"decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]",f32,f32,f64,"decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]"
10464243,0,null,0,"""F""","""UNK""","""UNK""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,69.0,172.333328,25.446404,-100.00000,81.00000,86.00000,-100.00000,128.00000,132.00000
2245024,2,null,0,"""F""","""05""","""N""","""UNK""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.0,-100.0,-100.0,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000
8474922,3,null,0,"""F""","""03""","""N""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,68.0,210.0,31.926904,-100.00000,70.00000,84.00000,-100.00000,122.00000,138.00000
7621798,0,null,0,"""F""","""05""","""Y""","""W""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,59.5,135.800003,26.966289,-100.00000,80.00000,80.00000,-100.00000,117.00000,117.00000
10756244,0,null,0,"""M""","""05""","""N""","""M""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-10

In [26]:
temp_materialized_df = temp_df.collect()

# Define the columns to one-hot encode
columns_to_one_hot_encode = ['Sex_CD', 'Race_CD', 'Hispanic_CD', 'Gender_CD']

# Combine the non-encoded columns with the encoded ones
dt_modeling_cat_encoded_df = pl.concat([
    temp_materialized_df.select(pl.all().exclude(columns_to_one_hot_encode)),
    temp_materialized_df.select(columns_to_one_hot_encode).to_dummies()
], how='horizontal')

In [27]:
dt_modeling_cat_encoded_df.head()

PATIENT_NUM,Age_group,FirstOutcomeDate,Outcome,nitrofurantoin,nystatin,atorvastatin,amlodipine,lisinopril_hydrochlorothiazide,tadalafil,albuterol,chlorthalidone,potassium chloride,vilanterol_fluticasone,hydrocortisone,lisinopril,"sennosides, USP_docusate",mupirocin,folic acid,tamsulosin,meloxicam,amitriptyline,quetiapine,gabapentin,tranexamic acid,polyvinyl alcohol,terbinafine,ascorbic acid,ferrous sulfate,losartan,clavulanate_amoxicillin,penicillin V,duloxetine,clobetasol,methadone,bupropion,tretinoin,…,LOINC:14805-6,LOINC:13056-7,LOINC:20567-4,LOINC:6942-7,LOINC:1764-0,LOINC:41649-5,LOINC:50210-4,LOINC:83102-4,LOINC:72888-1,LOINC:2349-9,LOINC:2657-5,mode_height,average_weight,BMI,average_diastolic_value,min_diastolic_value,max_diastolic_value,average_systolic_value,min_systolic_value,max_systolic_value,Sex_CD_F,Sex_CD_M,Sex_CD_UNK,Race_CD_01,Race_CD_02,Race_CD_03,Race_CD_04,Race_CD_05,Race_CD_UNK,Hispanic_CD_N,Hispanic_CD_UNK,Hispanic_CD_Y,Gender_CD_GQ,Gender_CD_M,Gender_CD_TG,Gender_CD_UNK,Gender_CD_W
i64,i64,datetime[μs],i64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,"decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]",f32,f32,f64,"decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]","decimal[18,5]",u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8
10464243,0,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,69.0,172.333328,25.446404,-100.00000,81.00000,86.00000,-100.00000,128.00000,132.00000,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1
2245024,2,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.0,-100.0,-100.0,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0
8474922,3,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,68.0,210.0,31.926904,-100.00000,70.00000,84.00000,-100.00000,122.00000,138.00000,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1
7621798,0,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,59.5,135.800003,26.966289,-100.00000,80.00000,80.00000,-100.00000,117.00000,117.00000,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
10756244,0,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,-100.00000,71.0,199.0,27.751835,-100.00000,86.00000,96.00000,-100.00000,132.00000,143.00000,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0


In [28]:
# drop unk column in demogrpahics 'Sex_CD', 'Race_CD', 'Hispanic_CD', 'Gender_CD'
'Hispanic_CD_UNK' in dt_modeling_cat_encoded_df.columns

True

In [29]:
dt_modeling_cat_encoded_df = dt_modeling_cat_encoded_df.drop('Sex_CD_UNK','Race_CD_UNK','Hispanic_CD_UNK','Gender_CD_UNK')

In [30]:
[col for col in dt_modeling_cat_encoded_df.columns if col.startswith('Gender')]

['Gender_CD_GQ', 'Gender_CD_M', 'Gender_CD_TG', 'Gender_CD_W']

In [31]:
if 'without_icd10z' in config.diag_path:
    diag_prefix = 'without_icd10z'
else:
    diag_prefix = 'with_icd10z'

if config.use_bmi_bp and config.use_cvs:
    dt_modeling_cat_encoded_df.write_parquet('./Determine_final_modeling_datasets/Determine_joined_med_'+'usage'
                      +'_lab_'+'latest'
                      +'_diag_'+'phe_'+diag_prefix
                      +'_bmi_bp_'+ 'cvs'              
                      +'_ordinal_nominal_encoded'+
                      '.parquet')

elif config.use_bmi_bp:
    dt_modeling_cat_encoded_df.write_parquet('./Determine_final_modeling_datasets/Determine_joined_med_'+'usage'
                      +'_lab_'+'latest'
                      +'_diag_'+'phe_'+diag_prefix
                      +'_bmi_bp_'              
                      +'_ordinal_nominal_encoded'+
                      '.parquet')


else:
    dt_modeling_cat_encoded_df.write_parquet('./Determine_final_modeling_datasets/Determine_joined_med_'+'usage'
                      +'_lab_'+'latest'
                      +'_diag_'+'phe_'+diag_prefix
                      +'_ordinal_nominal_encoded'+
                      '.parquet')

# Checking missing values for each patient

In [ ]:
modeling_df = pl.read_parquet('./Determine_final_modeling_datasets/Determine_joined_med_'+'usage'
                      +'_lab_'+'latest'
                      +'_diag_'+'phe_icd10'
                      +'_bmi_bp_'+ 'cvs'              
                      +'_ordinal_nominal_encoded'+
                      '.parquet')

In [ ]:
modeling_df = modeling_df.select([
    pl.col(col).cast(pl.Float32) if col.startswith('LOINC') else pl.col(col)
    for col in modeling_df.columns
])

In [ ]:
modeling_df.head()

In [ ]:
columns_to_ignore = [ 'Outcome', 'Age_group', 'FirstOutcomeDate'] + ['Sex_CD_F',
 'Sex_CD_M',
 #'Sex_CD_UNK',
 'Race_CD_01',
 'Race_CD_02',
 'Race_CD_03',
 'Race_CD_04',
 'Race_CD_05',
 #'Race_CD_06',
 #'Race_CD_07',
 #'Race_CD_UNK',
 'Hispanic_CD_N',
 #'Hispanic_CD_UNK',
 'Hispanic_CD_Y',
 'Gender_CD_GQ',
 'Gender_CD_M',
 'Gender_CD_TG',
 #'Gender_CD_UNK',
 'Gender_CD_W']

In [ ]:
modeling_df = modeling_df.drop(columns_to_ignore)

In [ ]:
lab_columns  = [col for col in modeling_df.columns if col.startswith('LOINC')]
med_diag_columns = list(set(modeling_df.columns) - set(lab_columns))

In [ ]:
print("Numbe of lab results columns", len(lab_columns))
print("Numbe of medications and diagnoses results columns", len(med_diag_columns))

In [ ]:
# sum_value_count_lab = []
# sum_value_count_med_diag = []
# for row in modeling_df.rows(named=True):
    
#     labs_dict = {key:row[key] for key in lab_columns}
#     med_diag_dict = {key:row[key] for key in med_diag_columns}
    
#     sum_value_count_lab.append(len(lab_columns) - list(labs_dict.values()).count(-100))
#     sum_value_count_med_diag.append(len(med_diag_columns) - list(med_diag_dict.values()).count(0))

In [ ]:
def count_values_in_med_diag(row:dict):
    return len(row) - list(row.values()).count(0)

def count_values_in_lab(row:dict):
    float_values = list(row.values())
    return len(float_values) - float_values.count(-100)

In [ ]:
counts_df = modeling_df.with_columns(
    pl.struct(pl.all().exclude(lab_columns)).map_elements(count_values_in_med_diag, return_dtype=pl.Int64).alias('non_null_med_diag_value_count'),
    pl.struct(pl.all().exclude(med_diag_columns)).map_elements(count_values_in_lab, return_dtype=pl.Int64).alias('non_null_lab_value_count')
    )

In [ ]:
counts_df.head()

In [ ]:
counts_df = counts_df.with_columns(pl.sum_horizontal(['non_null_med_diag_value_count','non_null_lab_value_count']).alias("Count_of_available_features"))

In [ ]:
counts_df.head()

In [ ]:
with open('catboost_false_positives_pids_test.pkl','rb') as f:
    fp_ids = pickle.load(f)

In [ ]:
import matplotlib.pyplot as plt
bins = 10 

data = counts_df['Count_of_available_features']

# Plotting the histogram
plt.xlim([min(data)-5, max(data)+5])
plt.hist(data, bins=bins, alpha=0.5)
# plt.xticks(range(10))
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Data points feature count')
plt.show()

## Check for key features missing rate in 